In [ ]:
# Get dataset single_speaker
!mkdir -p inputs/voices
!wget mayorx.xyz/Media/agentifai/large_audiobook_dataset.zip -O large_audiobook_dataset.zip
!unzip large_audiobook_dataset.zip
!rm large_audiobook_dataset.zip

!wget mayorx.xyz/Media/agentifai/input_voice.wav -O inputs/voices/input_voice.wav

Streaming output truncated to the last 5000 lines.
  inflating: dataset/wavs/A_Invencao_do_Dia_Claro_de_Almada_Negreiros_European_PT_segment_0524.wav  
  inflating: dataset/wavs/O_Defunto_de_Eca_de_Queiroz_European_PT_segment_0466.wav  
  inflating: dataset/wavs/Adao_e_Eva_no_Paraiso_de_Eca_de_Queiroz_European_PT_segment_0050.wav  
  inflating: dataset/wavs/O_Mandarim_de_Eca_de_Queiroz_European_PT_segment_0628.wav  
  inflating: dataset/wavs/O_Defunto_de_Eca_de_Queiroz_European_PT_segment_0089.wav  
  inflating: dataset/wavs/Poemas_de_Luiz_Vaz_de_Camoes_European_PT_segment_0678.wav  
  inflating: dataset/wavs/Adao_e_Eva_no_Paraiso_de_Eca_de_Queiroz_European_PT_segment_0670.wav  
  inflating: dataset/wavs/O_Mandarim_de_Eca_de_Queiroz_European_PT_segment_2556.wav  
  inflating: dataset/wavs/O_Mandarim_de_Eca_de_Queiroz_European_PT_segment_2925.wav  
  inflating: dataset/wavs/O_Mandarim_de_Eca_de_Queiroz_European_PT_segment_0774.wav  
  inflating: dataset/wavs/Livro_de_Magoas_de_Florbela_

In [ ]:
!apt install espeak-ng ffmpeg

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
The following additional packages will be installed:
  espeak-ng-data libespeak-ng1 libpcaudio0 libsonic0
The following NEW packages will be installed:
  espeak-ng espeak-ng-data libespeak-ng1 libpcaudio0 libsonic0
0 upgraded, 5 newly installed, 0 to remove and 49 not upgraded.
Need to get 4,526 kB of archives.
After this operation, 11.9 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libpcaudio0 amd64 1.1-6build2 [8,956 B]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libsonic0 amd64 0.2.0-11build1 [10.3 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 espeak-ng-data amd64 1.50+dfsg-10ubuntu0.1 [3,956 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libespeak-ng1 amd64 1.50+dfsg-10ubuntu0.1 [207 kB]
Get:5 http://archive.ubuntu.co

In [ ]:
!rm -rf coqui-ai-TTS
!git clone https://github.com/idiap/coqui-ai-TTS
!pip install -e coqui-ai-TTS/.
#!pip install coqui-tts

Cloning into 'coqui-ai-TTS'...
remote: Enumerating objects: 31054, done.
remote: Counting objects: 100% (3051/3051), done.
remote: Compressing objects: 100% (388/388), done.
remote: Total 31054 (delta 2770), reused 2787 (delta 2657), pack-reused 28003 (from 1)
Receiving objects: 100% (31054/31054), 137.00 MiB | 38.50 MiB/s, done.
Resolving deltas: 100% (22425/22425), done.
Obtaining file:///content/coqui-ai-TTS
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 26.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.3/85.3 kB 7.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 10.3 MB/s eta 0:00:00
  Preparin

In [ ]:
!mkdir -p run/training/XTTS_v2.0_original_model_files/
!wget https://huggingface.co/coqui/XTTS-v2/resolve/main/dvae.pth -O run/training/XTTS_v2.0_original_model_files/dvae.pth
!wget https://huggingface.co/coqui/XTTS-v2/resolve/main/mel_stats.pth -O run/training/XTTS_v2.0_original_model_files/mel_stats.pth
!wget https://huggingface.co/coqui/XTTS-v2/resolve/main/vocab.json -O run/training/XTTS_v2.0_original_model_files/vocab.json
#!wget https://huggingface.co/coqui/XTTS-v2/resolve/main/model.pth -O run/training/XTTS_v2.0_original_model_files/model.pth
!wget mayorx.xyz/Media/agentifai/model/best_model.pth -O run/training/XTTS_v2.0_original_model_files/model.pth

--2024-11-22 18:34:58--  https://huggingface.co/coqui/XTTS-v2/resolve/main/dvae.pth
Resolving huggingface.co (huggingface.co)... 18.172.134.88, 18.172.134.24, 18.172.134.4, ...
Connecting to huggingface.co (huggingface.co)|18.172.134.88|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/3e/73/3e73be38940b0ce7d10274a3412defcd0d3c9ea879d08bc8761d46cf6bbde3fa/b29bc227d410d4991e0a8c09b858f77415013eeb9fba9650258e96095557d97a?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27dvae.pth%3B+filename%3D%22dvae.pth%22%3B&Expires=1732559698&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczMjU1OTY5OH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzNlLzczLzNlNzNiZTM4OTQwYjBjZTdkMTAyNzRhMzQxMmRlZmNkMGQzYzllYTg3OWQwOGJjODc2MWQ0NmNmNmJiZGUzZmEvYjI5YmMyMjdkNDEwZDQ5OTFlMGE4YzA5Yjg1OGY3NzQxNTAxM2VlYjlmYmE5NjUwMjU4ZTk2MDk1NTU3ZDk3YT9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoifV19&S

In [ ]:
import sys
sys.path.append('/content/coqui-ai-TTS')

import os

from trainer import Trainer, TrainerArgs

from TTS.config.shared_configs import BaseDatasetConfig
from TTS.tts.datasets import load_tts_samples
from TTS.tts.layers.xtts.trainer.gpt_trainer import GPTArgs, GPTTrainer, GPTTrainerConfig, XttsAudioConfig
from TTS.utils.manage import ModelManager

In [ ]:


# Logging parameters
RUN_NAME = "GPT_XTTS_v2.0_LJSpeech_FT_PT"
PROJECT_NAME = "XTTS_trainer"
DASHBOARD_LOGGER = "tensorboard"
LOGGER_URI = None

# Set here the path that the checkpoints will be saved. Default: ./run/training/
OUT_PATH = os.path.join("run", "training")

# Training Parameters
OPTIMIZER_WD_ONLY_ON_WEIGHTS = True  # for multi-gpu training please make it False
START_WITH_EVAL = True  # if True it will star with evaluation
BATCH_SIZE = 6  # set here the batch size
GRAD_ACUMM_STEPS = 48  # set here the grad accumulation steps
# Note: we recommend that BATCH_SIZE * GRAD_ACUMM_STEPS need to be at least 252 for more efficient training. You can increase/decrease BATCH_SIZE but then set GRAD_ACUMM_STEPS accordingly.

# Define here the dataset that you want to use for the fine-tuning on.
config_dataset = BaseDatasetConfig(
    formatter="ljspeech",
    dataset_name="audiobooks",
    path="/content/dataset",
    meta_file_train="/content/dataset/metadata.csv",
    language="pt",
)

# Add here the configs of the datasets
DATASETS_CONFIG_LIST = [config_dataset]

# Define the path where XTTS v2.0.1 files will be downloaded
CHECKPOINTS_OUT_PATH = os.path.join(OUT_PATH, "XTTS_v2.0_original_model_files/")
os.makedirs(CHECKPOINTS_OUT_PATH, exist_ok=True)


# DVAE files
DVAE_CHECKPOINT_LINK = "https://huggingface.co/coqui/XTTS-v2/resolve/main/dvae.pth"
MEL_NORM_LINK = "https://huggingface.co/coqui/XTTS-v2/resolve/main/mel_stats.pth"

# Set the path to the downloaded files
DVAE_CHECKPOINT = os.path.join(CHECKPOINTS_OUT_PATH, os.path.basename(DVAE_CHECKPOINT_LINK))
MEL_NORM_FILE = os.path.join(CHECKPOINTS_OUT_PATH, os.path.basename(MEL_NORM_LINK))

# download DVAE files if needed
if not os.path.isfile(DVAE_CHECKPOINT) or not os.path.isfile(MEL_NORM_FILE):
    print(" > Downloading DVAE files!")
    ModelManager._download_model_files([MEL_NORM_LINK, DVAE_CHECKPOINT_LINK], CHECKPOINTS_OUT_PATH, progress_bar=True)


# Download XTTS v2.0 checkpoint if needed
TOKENIZER_FILE_LINK = "https://huggingface.co/coqui/XTTS-v2/resolve/main/vocab.json"

#XTTS_CHECKPOINT_LINK = "https://huggingface.co/coqui/XTTS-v2/resolve/main/model.pth"
XTTS_CHECKPOINT_LINK = "mayorx.xyz/Media/agentifai/model.pth"

# XTTS transfer learning parameters: You we need to provide the paths of XTTS model checkpoint that you want to do the fine tuning.
TOKENIZER_FILE = os.path.join(CHECKPOINTS_OUT_PATH, os.path.basename(TOKENIZER_FILE_LINK))  # vocab.json file
XTTS_CHECKPOINT = os.path.join(CHECKPOINTS_OUT_PATH, os.path.basename(XTTS_CHECKPOINT_LINK))  # model.pth file

# download XTTS v2.0 files if needed
if not os.path.isfile(TOKENIZER_FILE) or not os.path.isfile(XTTS_CHECKPOINT):
    print(" > Downloading XTTS v2.0 files!")
    ModelManager._download_model_files(
        [TOKENIZER_FILE_LINK, XTTS_CHECKPOINT_LINK], CHECKPOINTS_OUT_PATH, progress_bar=True
    )


# Training sentences generations
SPEAKER_REFERENCE = [
    "/content/inputs/voices/input_voice.wav"  # speaker reference to be used in training test sentences
]
LANGUAGE = config_dataset.language


def main():
    # init args and config
    model_args = GPTArgs(
        max_conditioning_length=220500,  # 10 secs
        min_conditioning_length=22050,  # 1 secs
        debug_loading_failures=False,
        max_wav_length=255995,  # ~11.6 seconds
        max_text_length=256,
        mel_norm_file=MEL_NORM_FILE,
        dvae_checkpoint=DVAE_CHECKPOINT,
        xtts_checkpoint=XTTS_CHECKPOINT,  # checkpoint path of the model that you want to fine-tune
        tokenizer_file=TOKENIZER_FILE,
        gpt_num_audio_tokens=1026,
        gpt_start_audio_token=1024,
        gpt_stop_audio_token=1025,
        gpt_use_masking_gt_prompt_approach=True,
        gpt_use_perceiver_resampler=True,
    )
    # define audio config
    audio_config = XttsAudioConfig(sample_rate=22050, dvae_sample_rate=22050, output_sample_rate=22050)
    # training parameters config
    config = GPTTrainerConfig(
        epochs=300,
        output_path=OUT_PATH,
        model_args=model_args,
        run_name=RUN_NAME,
        project_name=PROJECT_NAME,
        run_description="""
            GPT XTTS training
            """,
        dashboard_logger=DASHBOARD_LOGGER,
        logger_uri=LOGGER_URI,
        audio=audio_config,
        batch_size=BATCH_SIZE,
        batch_group_size=48,
        eval_batch_size=BATCH_SIZE,
        num_loader_workers=8,
        eval_split_max_size=256,
        print_step=100,
        plot_step=100,
        log_model_step=1000,
        save_step=10000,
        save_n_checkpoints=1,
        save_checkpoints=True,
        # target_loss="loss",
        print_eval=False,
        use_phonemes=True,
        phoneme_language="pt",
        phonemizer="espeak-ng",
        # Optimizer values like tortoise, pytorch implementation with modifications to not apply WD to non-weight parameters.
        optimizer="radam",
        optimizer_wd_only_on_weights=OPTIMIZER_WD_ONLY_ON_WEIGHTS,
        optimizer_params={"betas": [0.9, 0.96], "eps": 1e-8, "weight_decay": 1e-2},
        lr=5e-06,  # learning rate
        lr_scheduler="MultiStepLR",
        # it was adjusted accordly for the new step scheme
        lr_scheduler_params={"milestones": [50000 * 18, 150000 * 18, 300000 * 18], "gamma": 0.5, "last_epoch": -1},
        test_sentences=[
          {
              "text": "Olá",
              "speaker_wav": SPEAKER_REFERENCE,
              "language": LANGUAGE
          },
          {
              "text": "Obrigado",
              "speaker_wav": SPEAKER_REFERENCE,
              "language": LANGUAGE
          },
          {
              "text": "Como estás",
              "speaker_wav": SPEAKER_REFERENCE,
              "language": LANGUAGE
          },
          {
              "text": "Até amanhã",
              "speaker_wav": SPEAKER_REFERENCE,
              "language": LANGUAGE
          },
          {
              "text": "Vamos ao cinema",
              "speaker_wav": SPEAKER_REFERENCE,
              "language": LANGUAGE
          },
          {
              "text": "Está um dia bonito hoje",
              "speaker_wav": SPEAKER_REFERENCE,
              "language": LANGUAGE
          },
          {
              "text": "Preciso de comprar pão na padaria",
              "speaker_wav": SPEAKER_REFERENCE,
              "language": LANGUAGE
          },
          {
              "text": "O cão correu atrás da bola no parque",
              "speaker_wav": SPEAKER_REFERENCE,
              "language": LANGUAGE
          },
          {
              "text": "Ela leu o livro todo numa tarde chuvosa",
              "speaker_wav": SPEAKER_REFERENCE,
              "language": LANGUAGE
          },
          {
              "text": "O professor explicou a lição com muita paciência",
              "speaker_wav": SPEAKER_REFERENCE,
              "language": LANGUAGE
          },
          {
              "text": "Eles decidiram viajar pelo país durante as férias",
              "speaker_wav": SPEAKER_REFERENCE,
              "language": LANGUAGE
          },
          {
              "text": "O sol brilhava intensamente no céu sem nuvens",
              "speaker_wav": SPEAKER_REFERENCE,
              "language": LANGUAGE
          },
          {
              "text": "No inverno gostamos de beber chá quente junto à lareira",
              "speaker_wav": SPEAKER_REFERENCE,
              "language": LANGUAGE
          },
          {
              "text": "A cidade estava cheia de luzes e decorações de Natal",
              "speaker_wav": SPEAKER_REFERENCE,
              "language": LANGUAGE
          },
          {
              "text": "Estudamos juntos na biblioteca para preparar o exame final",
              "speaker_wav": SPEAKER_REFERENCE,
              "language": LANGUAGE
          },
          {
              "text": "O concerto foi magnífico com músicos talentosos e melodias emocionantes",
              "speaker_wav": SPEAKER_REFERENCE,
              "language": LANGUAGE
          },
          {
              "text": "Depois de uma longa caminhada descansaram à sombra de uma árvore",
              "speaker_wav": SPEAKER_REFERENCE,
              "language": LANGUAGE
          },
          {
              "text": "Ela preparou uma surpresa especial para o aniversário do irmão",
              "speaker_wav": SPEAKER_REFERENCE,
              "language": LANGUAGE
          },
          {
              "text": "Os cientistas descobriram uma nova espécie de planta na floresta tropical",
              "speaker_wav": SPEAKER_REFERENCE,
              "language": LANGUAGE
          },
          {
              "text": "A exposição de arte contemporânea atraiu muitos visitantes este ano",
              "speaker_wav": SPEAKER_REFERENCE,
              "language": LANGUAGE
          },
          {
              "text": "Apesar da chuva decidiram continuar com o piquenique no parque",
              "speaker_wav": SPEAKER_REFERENCE,
              "language": LANGUAGE
          },
          {
              "text": "Os atletas treinaram arduamente para se prepararem para a competição",
              "speaker_wav": SPEAKER_REFERENCE,
              "language": LANGUAGE
          },
          {
              "text": "A biblioteca oferece uma vasta coleção de livros e recursos digitais",
              "speaker_wav": SPEAKER_REFERENCE,
              "language": LANGUAGE
          },
          {
              "text": "Durante a viagem aprenderam muito sobre as culturas locais e tradições",
              "speaker_wav": SPEAKER_REFERENCE,
              "language": LANGUAGE
          },
          {
              "text": "Com determinação e esforço alcançou todos os seus objetivos profissionais",
              "speaker_wav": SPEAKER_REFERENCE,
              "language": LANGUAGE
          },
          {
              "text": "No fim de semana planearam visitar a aldeia histórica e explorar as ruínas antigas",
              "speaker_wav": SPEAKER_REFERENCE,
              "language": LANGUAGE
          },
      ],
    )

    # init the model from config
    model = GPTTrainer.init_from_config(config)

    # load training samples
    train_samples, eval_samples = load_tts_samples(
        DATASETS_CONFIG_LIST,
        eval_split=True,
        eval_split_max_size=config.eval_split_max_size,
        eval_split_size=config.eval_split_size,
    )

    # init the trainer and 🚀
    trainer = Trainer(
        TrainerArgs(
            restore_path=None,  # xtts checkpoint is restored via xtts_checkpoint key so no need of restore it using Trainer restore_path parameter
            skip_train_epoch=False,
            start_with_eval=START_WITH_EVAL,
            grad_accum_steps=GRAD_ACUMM_STEPS,
        ),
        config,
        output_path=OUT_PATH,
        model=model,
        train_samples=train_samples,
        eval_samples=eval_samples,
    )
    trainer.fit()


if __name__ == "__main__":
    main()

 > Training Environment:
 | > Backend: Torch
 | > Mixed precision: False
 | > Precision: float32
 | > Current device: 0
 | > Num. of GPUs: 1
 | > Num. of CPUs: 12
 | > Num. of Torch Threads: 1
 | > Torch seed: 1
 | > Torch CUDNN: True
 | > Torch CUDNN deterministic: False
 | > Torch CUDNN benchmark: False
 | > Torch TF32 MatMul: False
 > Start Tensorboard: tensorboard --logdir=run/training/GPT_XTTS_v2.0_LJSpeech_FT_PT-November-22-2024_06+40PM-0000000

 > Model has 518442047 parameters

 > EPOCH: 0/250
 --> run/training/GPT_XTTS_v2.0_LJSpeech_FT_PT-November-22-2024_06+40PM-0000000

 > EVALUATION 


  --> EVAL PERFORMANCE
     | > avg_loader_time: 0.18350093504961798 (+0)
     | > avg_loss_text_ce: 0.02486465433064629 (+0)
     | > avg_loss_mel_ce: 2.581977802164414 (+0)
     | > avg_loss: 2.606842475778916 (+0)


 > EPOCH: 1/250
 --> run/training/GPT_XTTS_v2.0_LJSpeech_FT_PT-November-22-2024_06+40PM-0000000

 > TRAINING (2024-11-22 18:41:43) 

   --> TIME: 2024-11-22 18:41:46 -- STEP: 0

RuntimeError: [enforce fail at inline_container.cc:603] . unexpected pos 1664518528 vs 1664518416

In [ ]:
!zip -rv fine_tuned_model.zip run/training/GPT_XTTS*

  adding: run/training/GPT_XTTS_v2.0_LJSpeech_FT_PT-November-17-2024_10+49PM-0000000/	(in=0) (out=0) (stored 0%)
  adding: run/training/GPT_XTTS_v2.0_LJSpeech_FT_PT-November-17-2024_10+49PM-0000000/checkpoint_65.pth .....................................................................................................................................................................................................................................................................................................................................	(in=3402409408) (out=3147090563) (deflated 8%)
  adding: run/training/GPT_XTTS_v2.0_LJSpeech_FT_PT-November-17-2024_10+49PM-0000000/trainer_0_log.txt 	(in=504947) (out=118918) (deflated 76%)
  adding: run/training/GPT_XTTS_v2.0_LJSpeech_FT_PT-November-17-2024_10+49PM-0000000/config.json	(in=5999) (out=1856) (deflated 69%)
  adding: run/training/GPT_XTTS_v2.0_LJSpeech_FT_PT-November-17-2024_10+49PM-0000000/colab_kernel_launcher.py	(in=1364) (out=765) (def

In [ ]:
!cp fine_tuned_model.zip /content/drive/MyDrive/

In [ ]:
from google.colab import files
files.download('fine_tuned_model.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!rm -rf run/training/GPT_XTTS*